# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

In [1]:
! pip install -U langchain

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ---------------------------------------- 0.0/987.6 kB ? eta -:--:--
     ---------------------------------------- 10.2/987.6 kB ? eta -:--:--
     - ----------------------------------- 41.0/987.6 kB 487.6 kB/s eta 0:00:02
     ---- ------------------------------- 122.9/987.6 kB 901.1 kB/s eta 0:00:01
     --------- ---------------------------- 256.0/987.6 kB 1.4 MB/s eta 0:00:01
     -------------------- ----------------- 522.2/987.6 kB 2.3 MB/s eta 0:00:01
     -------------------------------------  983.0/987.6 kB 3.9 MB/s eta 0:00:01
     -------------------------------------- 987.6/987.6 kB 3.7 MB/s eta 0:00:00
     ---------------------------------------- 0.0/366.5 kB ? eta -:--:--
     ------------------------------------- 366.5/366.5 kB 23.8 MB/s eta 0:00:00
     ---------------------------------------- 0.0/129.4 kB ? eta -:--:--
     ---------------------------------------- 129.4/129.4 kB ? eta 0:00:00
  Attemptin

## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

## Router Chain: 实现条件判断的大模型调用


这段代码构建了一个可定制的链路系统，用户可以提供不同的输入提示，并根据这些提示获取适当的响应。

主要逻辑：从`prompt_infos`创建多个`LLMChain`对象，并将它们保存在一个字典中，然后创建一个默认的`ConversationChain`，最后创建一个带有路由功能的`MultiPromptChain`。

![](../images/router_chain.png)

In [2]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [19]:
physics_template = """你是一位非常聪明的物理教授。
你擅长以简洁易懂的方式回答关于物理的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""


math_template = """你是一位很棒的数学家。你擅长回答数学问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。



这是一个问题：
{input}"""

biology_template = """你是一位深谙生命奥秘的生物学家，擅长以形象生动的比喻揭示生物界错综复杂的运作机制。
你对科学的探索充满了智慧和洞见，同时保持着对未知的敬畏和不懈的好奇心，总是以谦逊的姿态迎接每一个新的科学发现。

这是一个问题：
{input}"""

computer_template = """你是一位精通算法奥秘的计算机学家，擅长用贴切的类比来阐述计算机科学中的复杂概念。
你对技术的追求充满了洞察力和前瞻性，同时保持着对未知领域的谦逊态度和永不满足的好奇心，总是以开放的心态迎接每一次技术革新的挑战。

这是一个问题：
{input}"""

chinese_literature_template = """你是一位深谙汉语文化精髓的汉语言文学家，擅长用生动的故事来解析复杂的语言现象。
你对文学的探索充满了洞察力和细腻的感知，同时保持着对语言奥妙的好奇心和谦逊的学习态度，总是以温文尔雅的姿态引领他人走进汉语言文学的博大精深。

这是一个问题：
{input}"""


In [32]:
prompt_infos = [
    {
        "name": "物理",
        "description": "适用于回答物理问题",
        "prompt_template": physics_template,
    },
    {
        "name": "数学",
        "description": "适用于回答数学问题",
        "prompt_template": math_template,
    },
    {
        "name": "生物",
        "description": "适用于回答生物问题",
        "prompt_template": biology_template,
    },
    {
        "name": "计算机",
        "description": "适用于回答计算机问题",
        "prompt_template": computer_template,
    },
    {
        "name": "汉语言文学",
        "description": "适用于回答汉语言文学问题",
        "prompt_template": chinese_literature_template,
    },
]

In [33]:
llm = OpenAI(model_name="gpt-3.5-turbo-instruct",base_url="https://api.xiaoai.plus/v1")

In [34]:
# 创建一个空的目标链字典，用于存放根据prompt_infos生成的LLMChain。
destination_chains = {}

# 遍历prompt_infos列表，为每个信息创建一个LLMChain。
for p_info in prompt_infos:
    name = p_info["name"]  # 提取名称
    prompt_template = p_info["prompt_template"]  # 提取模板
    # 创建PromptTemplate对象
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # 使用上述模板和llm对象创建LLMChain对象
    chain = LLMChain(llm=llm, prompt=prompt)
    # 将新创建的chain对象添加到destination_chains字典中
    destination_chains[name] = chain

# 创建一个默认的ConversationChain
default_chain = ConversationChain(llm=llm, output_key="text")

In [35]:
type(default_chain)

langchain.chains.conversation.base.ConversationChain

### 使用 LLMRouterChain 实现条件判断调用

这段代码定义了一个chain对象（LLMRouterChain），该对象首先使用router_chain来决定哪个destination_chain应该被执行，如果没有合适的目标链，则默认使用default_chain。

In [36]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [37]:
# 从prompt_infos中提取目标信息并将其转化为字符串列表
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)
# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
# 创建路由的PromptTemplate
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [38]:
print(destinations)

['物理: 适用于回答物理问题', '数学: 适用于回答数学问题', '生物: 适用于回答生物问题', '计算机: 适用于回答计算机问题', '汉语言文学: 适用于回答汉语言文学问题']


In [39]:
print(destinations_str)

物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
计算机: 适用于回答计算机问题
汉语言文学: 适用于回答汉语言文学问题


In [40]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [41]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
计算机: 适用于回答计算机问题
汉语言文学: 适用于回答汉语言文学问题

<< INPUT >>
{input}

<

In [42]:
# 创建MultiPromptChain对象，其中包含了路由链，目标链和默认链。
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [43]:
print(chain.invoke("黑体辐射是什么？?"))



> Entering new MultiPromptChain chain...
物理: {'input': '黑体辐射是什么？?'}
> Finished chain.
{'input': '黑体辐射是什么？?', 'text': '\n\n黑体辐射是一种理想化的物理现象，它指的是在温度为绝对零度时，物体所发出的电磁辐射。这种辐射的光谱分布与物体的温度有关，符合普朗克定律。黑体辐射在物理学中有很重要的作用，能够解释一些重要的现象，如热力学平衡、宇宙微波背景辐射等。'}


In [44]:
print(
    chain.invoke(
        "大于40的第一个质数是多少，使得这个质数加一能被3整除？"
    )
)



> Entering new MultiPromptChain chain...
数学: {'input': '40'}
> Finished chain.
{'input': '40', 'text': '\n这不是一个问题，这是一个数字。要成为一位优秀的数学家，你需要学习如何提出和解决问题，而不仅仅是回答给定的数字或方程式。数学是一门探索和解决问题的学科，它要求思考和创新能力。因此，要想成为一位优秀的数学家，你需要不断练习和挑战自己，学习如何应用数学知识来解决实际问题。'}


In [45]:
router_chain.verbose = True

In [47]:
print(chain.invoke("黑洞是什么？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
物理: {'input': '什么是黑洞？'}
> Finished chain.
{'input': '什么是黑洞？', 'text': '\n黑洞是一种极端密度的天体，它的引力非常强大，以至于连光也无法逃脱。它的存在是由于质量非常巨大的恒星在死亡时产生的。当恒星的质量超过一定的极限，它会发生坍缩，形成一个极端密度的物体，即黑洞。由于黑洞的引力非常强大，它可以吞噬附近的物质，并且会不断吸收周围的物质增加自身的质量。目前，人类还无法直接观测黑洞，但通过间接的观测和理论计算，我们可以推测它们的存在。黑洞是宇宙中最神秘和令人着迷的天体之一，它们的研究也为我们揭示'}


### Homework

#### 扩展 Demo：实现生物、计算机和汉语言文学老师 PromptTemplates 及对应 Chains

In [48]:
print(chain.invoke("如何在细胞层面上解释生物体的衰老过程？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
生物: {'input': '如何在细胞层面上解释生物体的衰老过程？'}
> Finished chain.
{'input': '如何在细胞层面上解释生物体的衰老过程？', 'text': '\n\n在细胞层面上，生物体的衰老过程可以解释为细胞功能逐渐衰退、损伤累积以及细胞死亡导致的。细胞的衰老可以分为两种类型：一种是自然衰老，即由于细胞内部的遗传信息不断累积错误导致细胞功能衰退；另一种是外在因素引起的衰老，比如环境因素、生活方式等。\n\n细胞衰老的主要原因之一是端粒缩短。端粒是位于染色体末端的一段DNA序列，它们的主要作用是保护染色体不受损伤。每当细胞分裂时，端粒都会缩短一小段，随着时间的推移，端粒变得越来'}


In [49]:
print(chain.invoke("如何优化机器学习算法以提高预测模型的准确性和效率？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
计算机: {'input': '如何优化机器学习算法'}
> Finished chain.
{'input': '如何优化机器学习算法', 'text': '的训练过程？\n\n这就像是一场马拉松比赛，机器学习算法就是参赛选手，而训练过程则是训练的训练路线。要想优化算法的训练过程，就需要从两个方向着手：一是提高选手的个人素质，二是优化训练的路线。\n\n首先，我们可以通过不断改进算法的设计和优化参数来提高选手的个人素质。就像一位运动员不断训练和调整自己的身体素质来提高自己的比赛水平一样，我们可以通过改善算法的结构和参数来提高其准确性和泛化能力。例如，使用更有效的模型架构、改进损失函数、选择更合适的优化器等等。\n\n其次，我们还可以通过优化训练的路线'}


In [50]:
print(chain.invoke("如何分析《红楼梦》中的人物语言风格，以及这些风格如何反映出角色的性格和社会地位？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
汉语言文学: {'input': '如何分析《红楼梦》中的人物语言风格，以及这些风格如何反映出角色的性格和社会地位？'}
> Finished chain.
{'input': '如何分析《红楼梦》中的人物语言风格，以及这些风格如何反映出角色的性格和社会地位？', 'text': '\n\n《红楼梦》是中国古典文学中的经典之作，其中的人物形象和语言风格都具有深刻的文学意义。要分析《红楼梦》中的人物语言风格，首先需要对这部作品的背景和特点有一定的了解。\n\n《红楼梦》是一部具有浓厚社会色彩的长篇小说，它刻画了清朝贵族社会的生活和人物形象。在这个社会中，人们的身份地位和社会关系非常复杂，语言也成为了彰显身份和表达情感的重要手段。\n\n首先，我们可以从人物的语言习惯和用词特点来分析其语言风格。比如，贾宝玉作为贾府的少爷，他的语言中充满'}
